<a href="https://colab.research.google.com/github/LaboraDev/hackathon_one_flightOnTime/blob/main/pipeline_AnaAmelia_teste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

"""
================================================================================
SEMANA 2: PIPELINE DE PRÉ-PROCESSAMENTO E FEATURE ENGINEERING
================================================================================

Responsáveis:
- DS1 (Pré-processamento): Ana
- DS2 (Feature Engineering): Amélia

Data: 23/12/2025

Objetivo:
Criar pipeline reutilizável que funcione em dois cenários:
1. Treinamento: DataFrame completo (milhões de linhas)
2. API/Produção: 1 voo por vez (JSON)

Problema Identificado na Semana 1:
- Features criadas com groupby() não funcionam na API
- Transformações precisam estar DENTRO do pipeline

Solução:
- Custom Transformer para features agregadas (médias)
- Pipeline sklearn para normalização e encoding
- Funções reutilizáveis para features temporais

Entregáveis:
- media_transformer_ds2.pkl (DS2)
- preprocessor_ds1.pkl (DS1)
- documentacao_ds1_ds2.json
- Dados processados para DS3 treinar modelo

================================================================================
"""

print("="*80)
print("SEMANA 2 - DS1 + DS2: PIPELINE UNIFICADO")
print("="*80)


In [ ]:


import pandas as pd
import numpy as np
import pickle
import json
from datetime import datetime

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin

# Warnings
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliotecas importadas com sucesso!")


✅ Bibliotecas importadas com sucesso!


In [ ]:
# ============================================
# CARREGAR DADOS DA SEMANA 1 (DO DRIVE)
# ============================================

import gdown
import os
import zipfile
import glob

print("Carregando dados do Google Drive...")

file_id = "1207psedBKvnS0pJkDITroSzPiWrcz0ag"
zip_path = "dados_vra.zip"
extract_folder = "dados_vra"

if not os.path.exists(zip_path):
    url = f"https://drive.google.com/uc?id={file_id}"
    print("Baixando arquivo do Drive...")
    gdown.download(url, zip_path, quiet=False)
else:
    print("Arquivo ZIP ja existe")

if not os.path.exists(extract_folder):
    print("Extraindo arquivos...")
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(extract_folder)
else:
    print("Arquivos ja extraidos")

def carregar_vra(pasta, padrao="VRA_*.csv", sep=";", encoding="latin-1", skiprows=1):
    caminho_busca = os.path.join(pasta, padrao)
    arquivos = sorted(glob.glob(caminho_busca))
    if not arquivos:
        raise FileNotFoundError(f"Nenhum arquivo encontrado em {caminho_busca}")
    print(f"{len(arquivos)} arquivos encontrados")
    dfs = []
    for arquivo in arquivos:
        df_temp = pd.read_csv(arquivo, sep=sep, encoding=encoding, skiprows=skiprows)
        dfs.append(df_temp)
    df_final = pd.concat(dfs, ignore_index=True)
    print(f"Concatenacao concluida: {df_final.shape}")
    return df_final

print("\nCarregando arquivos CSV...")
df = carregar_vra(pasta="/content/dados_vra/dados_vra")

print("\nPadronizando nomes das colunas...")
df.columns = [c.encode("latin1").decode("utf-8") for c in df.columns]
mapa_colunas = {
    "ICAO Empresa Aérea": "empresa_aerea",
    "Número Voo": "numero_voo",
    "Código Autorização (DI)": "codigo_autorizacao_di",
    "Código Tipo Linha": "codigo_tipo_linha",
    "ICAO Aeródromo Origem": "aerodromo_origem",
    "ICAO Aeródromo Destino": "aerodromo_destino",
    "Partida Prevista": "partida_prevista",
    "Partida Real": "partida_real",
    "Chegada Prevista": "chegada_prevista",
    "Chegada Real": "chegada_real",
    "Situação Voo": "situacao_voo",
    "Código Justificativa": "codigo_justificativa"
}
df = df.rename(columns=mapa_colunas)

print("Convertendo datas...")
df["partida_prevista"] = pd.to_datetime(df["partida_prevista"], format="%Y-%m-%d %H:%M:%S", errors='coerce')
df["partida_real"] = pd.to_datetime(df["partida_real"], format="%Y-%m-%d %H:%M:%S", errors='coerce')

print("Criando flags de qualidade...")
df["flag_partida_prevista_ausente"] = df["partida_prevista"].isna()
df["flag_partida_real_ausente"] = df["partida_real"].isna()
df["flag_aerodromo_origem_ausente"] = df["aerodromo_origem"].isna()
df["flag_data_partida_fora_periodo"] = (
    df["partida_prevista"].notna() &
    ((df["partida_prevista"].dt.year < 2021) | (df["partida_prevista"].dt.year > 2025))
)
LIMITE_HORAS = 24
delta_h = (df["partida_real"] - df["partida_prevista"]).dt.total_seconds() / 3600
df["flag_partida_muito_alto"] = (
    df["partida_prevista"].notna() &
    df["partida_real"].notna() &
    (delta_h.abs() > LIMITE_HORAS)
)

print("Criando variavel alvo...")
def classificar_situacao_partida(minutos):
    if minutos < 0:
        return "Antecipado"
    elif minutos <= 15:
        return "Pontual"
    elif minutos <= 60:
        return "Atraso 15-60"
    elif minutos <= 120:
        return "Atraso 60-120"
    elif minutos <= 240:
        return "Atraso 120-240"
    else:
        return "Atraso > 240"

voos_rotulavel = (
    ~df["flag_partida_prevista_ausente"] &
    ~df["flag_partida_real_ausente"] &
    ~df["flag_data_partida_fora_periodo"] &
    ~df["flag_partida_muito_alto"]
)
df["atraso_partida_min"] = ((df["partida_real"] - df["partida_prevista"]).dt.total_seconds() / 60)
df["situacao_partida"] = pd.Series(index=df.index, dtype="object")
df.loc[voos_rotulavel, "situacao_partida"] = df.loc[voos_rotulavel, "atraso_partida_min"].apply(classificar_situacao_partida)
df = df[df["situacao_partida"].notna()].copy()
df['atrasado'] = (df['atraso_partida_min'] > 15).astype(int)

print("\n" + "="*80)
print("DATASET CARREGADO E PROCESSADO COM SUCESSO")
print("="*80)
print(f"Shape: {df.shape}")
print(f"Distribuicao: Pontuais={100*(df['atrasado']==0).mean():.1f}%, Atrasados={100*(df['atrasado']==1).mean():.1f}%")
print(f"\nColunas disponiveis:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col}")


Carregando dados do Google Drive...
Arquivo ZIP ja existe
Arquivos ja extraidos

Carregando arquivos CSV...
54 arquivos encontrados
Concatenacao concluida: (3968418, 12)

Padronizando nomes das colunas...
Convertendo datas...
Criando flags de qualidade...
Criando variavel alvo...

DATASET CARREGADO E PROCESSADO COM SUCESSO
Shape: (3642571, 20)
Distribuicao: Pontuais=84.0%, Atrasados=16.0%

Colunas disponiveis:
   1. empresa_aerea
   2. numero_voo
   3. codigo_autorizacao_di
   4. codigo_tipo_linha
   5. aerodromo_origem
   6. aerodromo_destino
   7. partida_prevista
   8. partida_real
   9. chegada_prevista
  10. chegada_real
  11. situacao_voo
  12. codigo_justificativa
  13. flag_partida_prevista_ausente
  14. flag_partida_real_ausente
  15. flag_aerodromo_origem_ausente
  16. flag_data_partida_fora_periodo
  17. flag_partida_muito_alto
  18. atraso_partida_min
  19. situacao_partida
  20. atrasado


In [ ]:
# ============================================
# ANALISE: COLUNAS RELEVANTES PARA O MODELO
# ============================================

colunas_usar = {
    "empresa_aerea": "SIM - Algumas companhias tem mais atrasos historicamente",
    "aerodromo_origem": "SIM - Alguns aeroportos tem mais congestionamento",
    "aerodromo_destino": "SIM - Alguns destinos tem mais problemas operacionais",
    "codigo_tipo_linha": "SIM - Voos internacionais vs domesticos tem padroes diferentes",
    "partida_prevista": "SIM - Usaremos para extrair: hora, dia da semana, mes (features temporais)"
}

colunas_nao_usar = {
    "numero_voo": "NAO - Identificador unico, causa overfitting",
    "codigo_autorizacao_di": "NAO - Identificador administrativo, sem valor preditivo",
    "partida_real": "NAO - So existe DEPOIS do voo partir (nao temos na predicao)",
    "chegada_prevista": "NAO - Acontece DEPOIS da partida",
    "chegada_real": "NAO - Acontece DEPOIS da partida",
    "situacao_voo": "NAO - So sabemos DEPOIS do voo acontecer",
    "codigo_justificativa": "NAO - 100% nulo (coluna descontinuada pela ANAC)",
    "flag_partida_prevista_ausente": "NAO - Sempre False (ja filtramos na Semana 1)",
    "flag_partida_real_ausente": "NAO - Sempre False (ja filtramos na Semana 1)",
    "flag_aerodromo_origem_ausente": "NAO - Sempre False (ja filtramos na Semana 1)",
    "flag_data_partida_fora_periodo": "NAO - Sempre False (ja filtramos na Semana 1)",
    "flag_partida_muito_alto": "NAO - Sempre False (ja filtramos na Semana 1)",
    "atraso_partida_min": "NAO - So existe DEPOIS do voo partir (target leakage)",
    "situacao_partida": "NAO - So existe DEPOIS do voo partir (target leakage)"
}

target_variaveis = {
    "atrasado": "TARGET - 0=Pontual, 1=Atrasado (>15 min)"
}

print("COLUNAS PARA USAR NO MODELO:")
for col, motivo in colunas_usar.items():
    print(f"  {col:25s} -> {motivo}")

print("\nCOLUNAS QUE NAO DEVEM SER USADAS:")
for col, motivo in colunas_nao_usar.items():
    print(f"  {col:35s} -> {motivo}")

print("\nTARGET (Variavel a prever):")
for col, motivo in target_variaveis.items():
    print(f"  {col:25s} -> {motivo}")


COLUNAS PARA USAR NO MODELO:
  empresa_aerea             -> SIM - Algumas companhias tem mais atrasos historicamente
  aerodromo_origem          -> SIM - Alguns aeroportos tem mais congestionamento
  aerodromo_destino         -> SIM - Alguns destinos tem mais problemas operacionais
  codigo_tipo_linha         -> SIM - Voos internacionais vs domesticos tem padroes diferentes
  partida_prevista          -> SIM - Usaremos para extrair: hora, dia da semana, mes (features temporais)

COLUNAS QUE NAO DEVEM SER USADAS:
  numero_voo                          -> NAO - Identificador unico, causa overfitting
  codigo_autorizacao_di               -> NAO - Identificador administrativo, sem valor preditivo
  partida_real                        -> NAO - So existe DEPOIS do voo partir (nao temos na predicao)
  chegada_prevista                    -> NAO - Acontece DEPOIS da partida
  chegada_real                        -> NAO - Acontece DEPOIS da partida
  situacao_voo                        -> NAO - So

In [ ]:
# ============================================
# REGRA DE OURO PARA SELECIONAR FEATURES
# ============================================

print("\nREGRA DE OURO:")
print("="*80)
print("Pergunta: 'Essa informacao existe ANTES do voo partir?'")
print("="*80)

print("\nSIM - Podemos usar:")
print("  - empresa_aerea           -> Sabemos qual companhia")
print("  - aerodromo_origem        -> Sabemos de onde sai")
print("  - aerodromo_destino       -> Sabemos para onde vai")
print("  - partida_prevista        -> Sabemos horario programado")
print("  - codigo_tipo_linha       -> Sabemos tipo de voo")

print("\nNAO - NAO podemos usar (vazamento de informacao):")
print("  - partida_real            -> So sabemos DEPOIS que decolou")
print("  - chegada_real            -> So sabemos DEPOIS que pousou")
print("  - atraso_partida_min      -> So calculamos DEPOIS da partida")
print("  - situacao_partida        -> So sabemos DEPOIS da partida")
print("  - situacao_voo            -> So sabemos DEPOIS do voo")

print("\n" + "="*80)
print("IMPORTANTE: Usar colunas que so existem DEPOIS causa 'data leakage'")
print("O modelo vai ter 100% de acuracia no treino, mas 0% em producao!")
print("="*80)



REGRA DE OURO:
Pergunta: 'Essa informacao existe ANTES do voo partir?'

SIM - Podemos usar:
  - empresa_aerea           -> Sabemos qual companhia
  - aerodromo_origem        -> Sabemos de onde sai
  - aerodromo_destino       -> Sabemos para onde vai
  - partida_prevista        -> Sabemos horario programado
  - codigo_tipo_linha       -> Sabemos tipo de voo

NAO - NAO podemos usar (vazamento de informacao):
  - partida_real            -> So sabemos DEPOIS que decolou
  - chegada_real            -> So sabemos DEPOIS que pousou
  - atraso_partida_min      -> So calculamos DEPOIS da partida
  - situacao_partida        -> So sabemos DEPOIS da partida
  - situacao_voo            -> So sabemos DEPOIS do voo

IMPORTANTE: Usar colunas que so existem DEPOIS causa 'data leakage'
O modelo vai ter 100% de acuracia no treino, mas 0% em producao!


In [ ]:
# ============================================
# FEATURES DERIVADAS (CRIADAS PELO PIPELINE)
# ============================================

print("\nFEATURES DERIVADAS DE 'partida_prevista':")
print("="*80)

features_derivadas = {
    "hora_dia": "Hora de 0 a 23 (ex: 14 para voos as 14h)",
    "dia_semana": "0=Segunda, 6=Domingo",
    "mes_ano": "1=Janeiro, 12=Dezembro",
    "periodo_dia": "Manha/Tarde/Noite/Madrugada",
    "fim_de_semana": "1 se Sexta/Sabado/Domingo",
    "alta_temporada": "1 se Dezembro ou Julho"
}

for feat, descricao in features_derivadas.items():
    print(f"  {feat:20s} -> {descricao}")

print("\nFEATURES AGREGADAS (MediaAtrasoTransformer):")
print("="*80)

features_agregadas = {
    "media_atraso_empresa": "Media historica de atraso da companhia",
    "media_atraso_origem": "Media historica de atraso do aeroporto origem",
    "media_atraso_destino": "Media historica de atraso do aeroporto destino"
}

for feat, descricao in features_agregadas.items():
    print(f"  {feat:25s} -> {descricao}")

print("\n" + "="*80)
print(f"TOTAL DE FEATURES PARA O MODELO: {len(colunas_usar) + len(features_derivadas) + len(features_agregadas)}")
print("="*80)



FEATURES DERIVADAS DE 'partida_prevista':
  hora_dia             -> Hora de 0 a 23 (ex: 14 para voos as 14h)
  dia_semana           -> 0=Segunda, 6=Domingo
  mes_ano              -> 1=Janeiro, 12=Dezembro
  periodo_dia          -> Manha/Tarde/Noite/Madrugada
  fim_de_semana        -> 1 se Sexta/Sabado/Domingo
  alta_temporada       -> 1 se Dezembro ou Julho

FEATURES AGREGADAS (MediaAtrasoTransformer):
  media_atraso_empresa      -> Media historica de atraso da companhia
  media_atraso_origem       -> Media historica de atraso do aeroporto origem
  media_atraso_destino      -> Media historica de atraso do aeroporto destino

TOTAL DE FEATURES PARA O MODELO: 14


In [ ]:
# ============================================
# MODELO: CLASSIFICACAO BINARIA COM PROBABILIDADES
# ============================================

print("\nTIPO DE MODELO:")
print("="*80)
print("Classificacao Binaria:")
print("  - Classe 0: Voo Pontual (atraso <= 15 min)")
print("  - Classe 1: Voo Atrasado (atraso > 15 min)")

print("\nOUTPUT DO MODELO:")
print("  1. Predicao: 0 ou 1 (classe prevista)")
print("  2. Probabilidade: 0.0 a 1.0 (confianca da predicao)")

print("\nEXEMPLO:")
print("  Input:  Voo TAM, SBGR -> SBSP, Segunda 14h")
print("  Output: Classe = 1 (Atrasado)")
print("          Probabilidade = 0.73 (73% de chance de atraso)")

print("\nMODELOS QUE RETORNAM PROBABILIDADE:")
print("  - LogisticRegression     -> model.predict_proba(X)")
print("  - RandomForestClassifier -> model.predict_proba(X)")
print("  - XGBoost                -> model.predict_proba(X)")
print("  - LightGBM               -> model.predict_proba(X)")

print("\n" + "="*80)
print("DS3 vai treinar modelo que retorna CLASSE e PROBABILIDADE")
print("="*80)



TIPO DE MODELO:
Classificacao Binaria:
  - Classe 0: Voo Pontual (atraso <= 15 min)
  - Classe 1: Voo Atrasado (atraso > 15 min)

OUTPUT DO MODELO:
  1. Predicao: 0 ou 1 (classe prevista)
  2. Probabilidade: 0.0 a 1.0 (confianca da predicao)

EXEMPLO:
  Input:  Voo TAM, SBGR -> SBSP, Segunda 14h
  Output: Classe = 1 (Atrasado)
          Probabilidade = 0.73 (73% de chance de atraso)

MODELOS QUE RETORNAM PROBABILIDADE:
  - LogisticRegression     -> model.predict_proba(X)
  - RandomForestClassifier -> model.predict_proba(X)
  - XGBoost                -> model.predict_proba(X)
  - LightGBM               -> model.predict_proba(X)

DS3 vai treinar modelo que retorna CLASSE e PROBABILIDADE


In [ ]:
# ============================================
# RESUMO: O QUE O MODELO VAI USAR
# ============================================

print("\n" + "="*80)
print("RESUMO FINAL: FEATURES PARA O MODELO")
print("="*80)

print("\nUSAREMOS (informacoes disponiveis ANTES do voo):")
print("  1. empresa_aerea          - Qual companhia")
print("  2. aerodromo_origem       - De onde sai")
print("  3. aerodromo_destino      - Para onde vai")
print("  4. codigo_tipo_linha      - Tipo de voo")
print("  5. hora_dia               - Que hora do dia")
print("  6. dia_semana             - Que dia da semana")
print("  7. mes_ano                - Que mes")
print("  8. periodo_dia            - Manha/Tarde/Noite/Madrugada")
print("  9. fim_de_semana          - Se e fim de semana")
print(" 10. alta_temporada         - Se e alta temporada")
print(" 11. media_atraso_empresa   - Historico da companhia")
print(" 12. media_atraso_origem    - Historico do aeroporto origem")
print(" 13. media_atraso_destino   - Historico do aeroporto destino")

print("\nNAO USAREMOS (informacoes so disponiveis DEPOIS do voo):")
print("  - partida_real")
print("  - chegada_real")
print("  - atraso_partida_min")
print("  - situacao_partida")
print("  - situacao_voo")

print("\nTARGET (o que queremos prever):")
print("  - atrasado (0=Pontual, 1=Atrasado)")

print("\nOUTPUT DO MODELO NA API:")
print("  - Classe: 0 ou 1")
print("  - Probabilidade: 0.0 a 1.0")

print("\n" + "="*80)



RESUMO FINAL: FEATURES PARA O MODELO

USAREMOS (informacoes disponiveis ANTES do voo):
  1. empresa_aerea          - Qual companhia
  2. aerodromo_origem       - De onde sai
  3. aerodromo_destino      - Para onde vai
  4. codigo_tipo_linha      - Tipo de voo
  5. hora_dia               - Que hora do dia
  6. dia_semana             - Que dia da semana
  7. mes_ano                - Que mes
  8. periodo_dia            - Manha/Tarde/Noite/Madrugada
  9. fim_de_semana          - Se e fim de semana
 10. alta_temporada         - Se e alta temporada
 11. media_atraso_empresa   - Historico da companhia
 12. media_atraso_origem    - Historico do aeroporto origem
 13. media_atraso_destino   - Historico do aeroporto destino

NAO USAREMOS (informacoes so disponiveis DEPOIS do voo):
  - partida_real
  - chegada_real
  - atraso_partida_min
  - situacao_partida
  - situacao_voo

TARGET (o que queremos prever):
  - atrasado (0=Pontual, 1=Atrasado)

OUTPUT DO MODELO NA API:
  - Classe: 0 ou 1
  - Prob

# Celulas de 4 á 16

##Celula 4


In [ ]:
# ============================================
# CRIAR AMOSTRA ESTRATIFICADA (10%)
# ============================================

print("Criando amostra estratificada...")
print("Motivo: Evitar explosao de memoria no OneHotEncoder")

df_sample, _ = train_test_split(
    df,
    test_size=0.90,
    random_state=42,
    stratify=df['atrasado']
)

print(f"\nAmostragem concluida:")
print(f"   Dataset original: {df.shape[0]:,} linhas")
print(f"   Amostra (10%):    {df_sample.shape[0]:,} linhas")
print(f"\n   Distribuicao preservada:")
print(f"   - Pontuais:  {(1-df_sample['atrasado'].mean())*100:.1f}%")
print(f"   - Atrasados: {df_sample['atrasado'].mean()*100:.1f}%")

del df
import gc
gc.collect()

print("\nMemoria liberada")


Criando amostra estratificada...
Motivo: Evitar explosao de memoria no OneHotEncoder

Amostragem concluida:
   Dataset original: 3,642,571 linhas
   Amostra (10%):    364,257 linhas

   Distribuicao preservada:
   - Pontuais:  84.0%
   - Atrasados: 16.0%

Memoria liberada


In [ ]:
# @title
# ============================================
# DS2: FEATURES TEMPORAIS (FUNCAO REUTILIZAVEL)
# ============================================

def criar_features_temporais(df):
    """
    Cria features derivadas de tempo a partir de 'partida_prevista'.

    Features criadas:
    - hora_dia: Hora de 0 a 23
    - dia_semana: 0=Segunda, 6=Domingo
    - mes_ano: 1=Janeiro, 12=Dezembro
    - periodo_dia: Manha/Tarde/Noite/Madrugada
    - fim_de_semana: 1 se Sexta/Sabado/Domingo
    - alta_temporada: 1 se Dezembro ou Julho
    """
    df = df.copy()

    df['hora_dia'] = df['partida_prevista'].dt.hour
    df['dia_semana'] = df['partida_prevista'].dt.dayofweek
    df['mes_ano'] = df['partida_prevista'].dt.month

    def classificar_periodo(hora):
        if 5 <= hora < 12:
            return 'Manha'
        elif 12 <= hora < 18:
            return 'Tarde'
        elif 18 <= hora < 22:
            return 'Noite'
        else:
            return 'Madrugada'

    df['periodo_dia'] = df['hora_dia'].apply(classificar_periodo)
    df['fim_de_semana'] = df['dia_semana'].isin([4, 5, 6]).astype(int)
    df['alta_temporada'] = df['mes_ano'].isin([7, 12]).astype(int)

    return df

print("Funcao criar_features_temporais() criada")
print("\nFeatures que serao criadas:")
print("  1. hora_dia (0-23)")
print("  2. dia_semana (0-6)")
print("  3. mes_ano (1-12)")
print("  4. periodo_dia (Manha/Tarde/Noite/Madrugada)")
print("  5. fim_de_semana (0/1)")
print("  6. alta_temporada (0/1)")


Funcao criar_features_temporais() criada

Features que serao criadas:
  1. hora_dia (0-23)
  2. dia_semana (0-6)
  3. mes_ano (1-12)
  4. periodo_dia (Manha/Tarde/Noite/Madrugada)
  5. fim_de_semana (0/1)
  6. alta_temporada (0/1)


In [ ]:
# ============================================
# DS2: CUSTOM TRANSFORMER PARA MEDIAS
# ============================================

class MediaAtrasoTransformer(BaseEstimator, TransformerMixin):
    """
    Transformer customizado para criar features de media de atraso.

    PROBLEMA RESOLVIDO:
    Na Semana 1, criamos features com groupby() direto no DataFrame.
    Isso NAO funciona na API porque a API recebe 1 voo por vez.

    SOLUCAO:
    - No fit(): Aprende as medias usando dados de treino
    - No transform(): Aplica medias aprendidas (funciona com 1 linha)
    """

    def __init__(self):
        self.medias_empresa = {}
        self.medias_origem = {}
        self.medias_destino = {}
        self.media_global = 0

    def fit(self, X, y=None):
        print("  [MediaAtrasoTransformer] Aprendendo medias...")

        self.medias_empresa = (
            X.groupby('empresa_aerea')['atraso_partida_min']
            .mean()
            .to_dict()
        )

        self.medias_origem = (
            X.groupby('aerodromo_origem')['atraso_partida_min']
            .mean()
            .to_dict()
        )

        self.medias_destino = (
            X.groupby('aerodromo_destino')['atraso_partida_min']
            .mean()
            .to_dict()
        )

        self.media_global = X['atraso_partida_min'].mean()

        print(f"  Medias aprendidas:")
        print(f"     - {len(self.medias_empresa)} empresas")
        print(f"     - {len(self.medias_origem)} aeroportos origem")
        print(f"     - {len(self.medias_destino)} aeroportos destino")
        print(f"     - Media global (fallback): {self.media_global:.2f} min")

        return self

    def transform(self, X):
        X = X.copy()

        X['media_atraso_empresa'] = (
            X['empresa_aerea']
            .map(self.medias_empresa)
            .fillna(self.media_global)
        )

        X['media_atraso_origem'] = (
            X['aerodromo_origem']
            .map(self.medias_origem)
            .fillna(self.media_global)
        )

        X['media_atraso_destino'] = (
            X['aerodromo_destino']
            .map(self.medias_destino)
            .fillna(self.media_global)
        )

        return X

print("MediaAtrasoTransformer criado")
print("\nPor que isso e importante?")
print("  - API recebe 1 voo por vez (nao tem acesso a outros voos)")
print("  - Transformer 'lembra' medias do treino e aplica em dados novos")
print("  - Funciona tanto em treino quanto em producao")


MediaAtrasoTransformer criado

Por que isso e importante?
  - API recebe 1 voo por vez (nao tem acesso a outros voos)
  - Transformer 'lembra' medias do treino e aplica em dados novos
  - Funciona tanto em treino quanto em producao


In [ ]:
# ============================================
# APLICAR FEATURES TEMPORAIS NO DATASET
# ============================================

print("Aplicando features temporais...")

df_sample = criar_features_temporais(df_sample)

print("Features temporais criadas")
print("\nExemplo das novas features:")
print(df_sample[['partida_prevista', 'hora_dia', 'periodo_dia',
                  'fim_de_semana', 'alta_temporada']].head(10))

print(f"\nShape atual: {df_sample.shape}")


Aplicando features temporais...
Features temporais criadas

Exemplo das novas features:
           partida_prevista  hora_dia periodo_dia  fim_de_semana  \
1619724 2023-10-29 17:20:00        17       Tarde              1   
3497825 2025-01-19 08:30:00         8       Manha              1   
2890320 2024-02-16 12:50:00        12       Tarde              1   
3457695 2024-09-18 14:15:00        14       Tarde              0   
2593466 2024-10-05 10:35:00        10       Manha              1   
1731633 2023-11-14 19:10:00        19       Noite              0   
2254243 2023-07-24 08:50:00         8       Manha              0   
2290712 2023-07-29 15:40:00        15       Tarde              1   
1235956 2022-06-06 11:25:00        11       Manha              0   
1382793 2022-08-18 07:40:00         7       Manha              0   

         alta_temporada  
1619724               0  
3497825               0  
2890320               0  
3457695               0  
2593466               0  
1731633

In [ ]:
# ============================================
# DS1: DEFINIR FEATURES PARA O PIPELINE
# ============================================

print("Definindo features para o pipeline...")

target = 'atrasado'

# Features originais que usaremos
features_originais = [
    'empresa_aerea',
    'aerodromo_origem',
    'aerodromo_destino',
    'codigo_tipo_linha'
]

# Features numericas base (criadas por criar_features_temporais)
numeric_features_base = [
    'hora_dia',
    'dia_semana',
    'mes_ano',
    'fim_de_semana',
    'alta_temporada'
]

# Features categoricas
categorical_features = [
    'empresa_aerea',
    'aerodromo_origem',
    'aerodromo_destino',
    'codigo_tipo_linha',
    'periodo_dia'
]

# Colunas necessarias para MediaAtrasoTransformer
# IMPORTANTE: atraso_partida_min so e usado para APRENDER as medias no treino
# Na API, nao precisaremos dessa coluna (usaremos as medias pre-calculadas)
colunas_para_medias = [
    'empresa_aerea',
    'aerodromo_origem',
    'aerodromo_destino',
    'atraso_partida_min'
]

print("\n" + "="*80)
print("FEATURES DEFINIDAS PARA O MODELO")
print("="*80)

print(f"\n1. FEATURES ORIGINAIS ({len(features_originais)}):")
for f in features_originais:
    print(f"   - {f}")

print(f"\n2. FEATURES NUMERICAS DERIVADAS ({len(numeric_features_base)}):")
for f in numeric_features_base:
    print(f"   - {f}")

print(f"\n3. FEATURES CATEGORICAS ({len(categorical_features)}):")
for f in categorical_features:
    print(f"   - {f}")

print(f"\n4. FEATURES AGREGADAS (criadas pelo MediaAtrasoTransformer):")
print("   - media_atraso_empresa")
print("   - media_atraso_origem")
print("   - media_atraso_destino")

print(f"\n" + "="*80)
print(f"TOTAL: {len(numeric_features_base) + 3 + len(categorical_features)} features")
print("="*80)

print("\nNOTA IMPORTANTE:")
print("  'atraso_partida_min' e usado APENAS para treinar o MediaAtrasoTransformer")
print("  Na API, nao precisaremos dessa coluna (usaremos medias pre-calculadas)")


Definindo features para o pipeline...

FEATURES DEFINIDAS PARA O MODELO

1. FEATURES ORIGINAIS (4):
   - empresa_aerea
   - aerodromo_origem
   - aerodromo_destino
   - codigo_tipo_linha

2. FEATURES NUMERICAS DERIVADAS (5):
   - hora_dia
   - dia_semana
   - mes_ano
   - fim_de_semana
   - alta_temporada

3. FEATURES CATEGORICAS (5):
   - empresa_aerea
   - aerodromo_origem
   - aerodromo_destino
   - codigo_tipo_linha
   - periodo_dia

4. FEATURES AGREGADAS (criadas pelo MediaAtrasoTransformer):
   - media_atraso_empresa
   - media_atraso_origem
   - media_atraso_destino

TOTAL: 13 features

NOTA IMPORTANTE:
  'atraso_partida_min' e usado APENAS para treinar o MediaAtrasoTransformer
  Na API, nao precisaremos dessa coluna (usaremos medias pre-calculadas)


In [ ]:
# ============================================
# SEPARAR X (FEATURES) E y (TARGET)
# ============================================

print("Separando features (X) e target (y)...")

colunas_necessarias = numeric_features_base + categorical_features + colunas_para_medias
colunas_necessarias = list(dict.fromkeys(colunas_necessarias))

X = df_sample[colunas_necessarias].copy()
y = df_sample[target].copy()

print(f"\nSeparacao concluida:")
print(f"   X shape: {X.shape}")
print(f"   y shape: {y.shape}")
print(f"\n   Distribuicao do target:")
print(f"   - Classe 0 (Pontual):  {(y==0).sum():,} ({(y==0).mean()*100:.1f}%)")
print(f"   - Classe 1 (Atrasado): {(y==1).sum():,} ({(y==1).mean()*100:.1f}%)")


Separando features (X) e target (y)...

Separacao concluida:
   X shape: (364257, 11)
   y shape: (364257,)

   Distribuicao do target:
   - Classe 0 (Pontual):  306,073 (84.0%)
   - Classe 1 (Atrasado): 58,184 (16.0%)


In [ ]:
# ============================================
# DS2: APLICAR MEDIAARASOTRANSFORMER
# ============================================

print("Aplicando MediaAtrasoTransformer...")
print("(Criando features: media_atraso_empresa, media_atraso_origem, media_atraso_destino)\n")

media_transformer = MediaAtrasoTransformer()
X_com_medias = media_transformer.fit_transform(X)

print(f"\nTransformer aplicado")
print(f"   Shape antes:  {X.shape}")
print(f"   Shape depois: {X_com_medias.shape}")
print(f"\nNovas colunas criadas:")
print(X_com_medias[['empresa_aerea', 'media_atraso_empresa',
                     'aerodromo_origem', 'media_atraso_origem',
                     'aerodromo_destino', 'media_atraso_destino']].head())


Aplicando MediaAtrasoTransformer...
(Criando features: media_atraso_empresa, media_atraso_origem, media_atraso_destino)

  [MediaAtrasoTransformer] Aprendendo medias...
  Medias aprendidas:
     - 126 empresas
     - 329 aeroportos origem
     - 328 aeroportos destino
     - Media global (fallback): 5.67 min

Transformer aplicado
   Shape antes:  (364257, 11)
   Shape depois: (364257, 14)

Novas colunas criadas:
        empresa_aerea  media_atraso_empresa aerodromo_origem  \
1619724           AZU              3.164909             SBRF   
3497825           AZU              3.164909             SBCF   
2890320           TAM              3.746490             SBCT   
3457695           TAM              3.746490             SBFZ   
2593466           AZU              3.164909             SBPS   

         media_atraso_origem aerodromo_destino  media_atraso_destino  
1619724             3.416646              SBSV              5.376899  
3497825             4.519614              SBCT           

In [ ]:
# ============================================
# DS1: PIPELINE DE PRE-PROCESSAMENTO
# ============================================

print("Criando pipeline de pre-processamento...")

numeric_features_final = numeric_features_base + [
    'media_atraso_empresa',
    'media_atraso_origem',
    'media_atraso_destino'
]

print(f"\n1. Pipeline para NUMERICAS ({len(numeric_features_final)} features):")
print("   - Imputer: Preencher nulos com mediana")
print("   - Scaler: Normalizar (media=0, std=1)")

print(f"\n2. Pipeline para CATEGORICAS ({len(categorical_features)} features):")
print("   - Imputer: Preencher nulos com 'DESCONHECIDO'")
print("   - Encoder: OrdinalEncoder (evita explosao de dimensao)")

preprocessor = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]), numeric_features_final),

    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value='DESCONHECIDO')),
        ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
    ]), categorical_features)
], remainder='drop')

print("\nPipeline criado")


Criando pipeline de pre-processamento...

1. Pipeline para NUMERICAS (8 features):
   - Imputer: Preencher nulos com mediana
   - Scaler: Normalizar (media=0, std=1)

2. Pipeline para CATEGORICAS (5 features):
   - Imputer: Preencher nulos com 'DESCONHECIDO'
   - Encoder: OrdinalEncoder (evita explosao de dimensao)

Pipeline criado


In [ ]:
# ============================================
# APLICAR PIPELINE COMPLETO
# ============================================

print("Aplicando pipeline de pre-processamento...\n")

X_processed = preprocessor.fit_transform(X_com_medias)

print(f"Pipeline aplicado com sucesso")
print(f"\n   Shape original:   {X.shape}")
print(f"   Shape processado: {X_processed.shape}")
print(f"\n   Tipo dos dados: {type(X_processed)}")
print(f"   Memoria usada: {X_processed.nbytes / 1024**2:.1f} MB")

print("\nDados prontos para treino do modelo")


Aplicando pipeline de pre-processamento...

Pipeline aplicado com sucesso

   Shape original:   (364257, 11)
   Shape processado: (364257, 13)

   Tipo dos dados: <class 'numpy.ndarray'>
   Memoria usada: 36.1 MB

Dados prontos para treino do modelo


In [ ]:
# ============================================
# SALVAR PIPELINES (ENTREGAVEIS)
# ============================================

print("Salvando pipelines...")

with open('media_transformer_ds2.pkl', 'wb') as f:
    pickle.dump(media_transformer, f)
print("Salvo: media_transformer_ds2.pkl")

with open('preprocessor_ds1.pkl', 'wb') as f:
    pickle.dump(preprocessor, f)
print("Salvo: preprocessor_ds1.pkl")

np.save('X_train_processed.npy', X_processed)
np.save('y_train.npy', y.values)
print("Salvo: X_train_processed.npy")
print("Salvo: y_train.npy")

print("\nTodos os arquivos salvos")
print("\nArquivos criados:")
print("  1. media_transformer_ds2.pkl")
print("  2. preprocessor_ds1.pkl")
print("  3. X_train_processed.npy")
print("  4. y_train.npy")


Salvando pipelines...
Salvo: media_transformer_ds2.pkl
Salvo: preprocessor_ds1.pkl
Salvo: X_train_processed.npy
Salvo: y_train.npy

Todos os arquivos salvos

Arquivos criados:
  1. media_transformer_ds2.pkl
  2. preprocessor_ds1.pkl
  3. X_train_processed.npy
  4. y_train.npy


In [ ]:
# ============================================
# TESTE: SIMULAR RECEBIMENTO DE NOVO VOO (API)
# ============================================

print("="*80)
print("TESTE: SIMULANDO RECEBIMENTO DE NOVO VOO VIA API")
print("="*80)

novo_voo_json = {
    'empresa_aerea': 'TAM',
    'aerodromo_origem': 'SBGR',
    'aerodromo_destino': 'SBSP',
    'partida_prevista': '2024-12-25 20:30:00',
    'atraso_partida_min': 0,
    'codigo_tipo_linha': 'N' # Adicionando a coluna faltante
}

print("\n1. JSON recebido pela API:")
print(json.dumps(novo_voo_json, indent=2, ensure_ascii=False))

novo_voo_df = pd.DataFrame([novo_voo_json])
novo_voo_df['partida_prevista'] = pd.to_datetime(novo_voo_df['partida_prevista'])

print("\n2. Aplicando criar_features_temporais()...")
novo_voo_df = criar_features_temporais(novo_voo_df)
print(f"   Features temporais criadas")

print("\n3. Aplicando MediaAtrasoTransformer...")
novo_voo_com_medias = media_transformer.transform(novo_voo_df)
print(f"   Features agregadas criadas:")
print(f"      - media_atraso_empresa: {novo_voo_com_medias['media_atraso_empresa'].values[0]:.2f} min")
print(f"      - media_atraso_origem:  {novo_voo_com_medias['media_atraso_origem'].values[0]:.2f} min")
print(f"      - media_atraso_destino: {novo_voo_com_medias['media_atraso_destino'].values[0]:.2f} min")

print("\n4. Aplicando Preprocessor (normalizacao + encoding)...")
novo_voo_processed = preprocessor.transform(novo_voo_com_medias)
print(f"   Shape final: {novo_voo_processed.shape}")

print("\n" + "="*80)
print("TESTE BEM-SUCEDIDO")
print("="*80)
print("\nO pipeline funciona corretamente com:")
print("  - DataFrame completo (milhoes de linhas)")
print("  - 1 voo individual (JSON da API)")
print("\nPronto para DS3 treinar modelo e DS5 integrar com API")


TESTE: SIMULANDO RECEBIMENTO DE NOVO VOO VIA API

1. JSON recebido pela API:
{
  "empresa_aerea": "TAM",
  "aerodromo_origem": "SBGR",
  "aerodromo_destino": "SBSP",
  "partida_prevista": "2024-12-25 20:30:00",
  "atraso_partida_min": 0,
  "codigo_tipo_linha": "N"
}

2. Aplicando criar_features_temporais()...
   Features temporais criadas

3. Aplicando MediaAtrasoTransformer...
   Features agregadas criadas:
      - media_atraso_empresa: 3.75 min
      - media_atraso_origem:  8.65 min
      - media_atraso_destino: 2.93 min

4. Aplicando Preprocessor (normalizacao + encoding)...
   Shape final: (1, 13)

TESTE BEM-SUCEDIDO

O pipeline funciona corretamente com:
  - DataFrame completo (milhoes de linhas)
  - 1 voo individual (JSON da API)

Pronto para DS3 treinar modelo e DS5 integrar com API


In [ ]:
# ============================================
# DOCUMENTACAO COMPLETA (DS1 + DS2)
# ============================================

documentacao = {
    "projeto": "Hackathon - Previsao de Atrasos de Voo",
    "semana": 2,
    "data": "23/12/2025",
    "responsaveis": {
        "DS1_Preprocessamento": "Ana",
        "DS2_FeatureEngineering": "Amelia"
    },
    "DS1_Preprocessamento": {
        "amostragem": {
            "motivo": "Evitar explosao de memoria no OneHotEncoder",
            "metodo": "Amostra estratificada de 10%",
            "distribuicao_preservada": True
        },
        "tratamento_nulos": {
            "numericas": "Mediana (SimpleImputer)",
            "categoricas": "Constante 'DESCONHECIDO' (SimpleImputer)"
        },
        "normalizacao": "StandardScaler (media=0, std=1)",
        "encoding": "OrdinalEncoder (evita explosao de dimensao)",
        "features_numericas": numeric_features_final,
        "features_categoricas": categorical_features
    },
    "DS2_FeatureEngineering": {
        "features_temporais": {
            "hora_dia": "0-23",
            "dia_semana": "0=Segunda, 6=Domingo",
            "mes_ano": "1-12",
            "periodo_dia": "Manha/Tarde/Noite/Madrugada",
            "fim_de_semana": "Sexta/Sabado/Domingo = 1",
            "alta_temporada": "Dezembro/Julho = 1"
        },
        "features_agregadas": {
            "media_atraso_empresa": "Media historica por companhia",
            "media_atraso_origem": "Media historica por aeroporto origem",
            "media_atraso_destino": "Media historica por aeroporto destino"
        }
    },
    "entregaveis": [
        "media_transformer_ds2.pkl",
        "preprocessor_ds1.pkl",
        "X_train_processed.npy",
        "y_train.npy",
        "documentacao_ds1_ds2.json"
    ],
    "proximo_passo": "DS3 treinar modelo baseline com X_train_processed.npy e y_train.npy"
}

with open('documentacao_ds1_ds2.json', 'w', encoding='utf-8') as f:
    json.dump(documentacao, f, indent=2, ensure_ascii=False)

print("Documentacao salva: documentacao_ds1_ds2.json")
print("\n" + "="*80)
print("RESUMO DA DOCUMENTACAO")
print("="*80)
print(json.dumps(documentacao, indent=2, ensure_ascii=False))


Documentacao salva: documentacao_ds1_ds2.json

RESUMO DA DOCUMENTACAO
{
  "projeto": "Hackathon - Previsao de Atrasos de Voo",
  "semana": 2,
  "data": "23/12/2025",
  "responsaveis": {
    "DS1_Preprocessamento": "Ana",
    "DS2_FeatureEngineering": "Amelia"
  },
  "DS1_Preprocessamento": {
    "amostragem": {
      "motivo": "Evitar explosao de memoria no OneHotEncoder",
      "metodo": "Amostra estratificada de 10%",
      "distribuicao_preservada": true
    },
    "tratamento_nulos": {
      "numericas": "Mediana (SimpleImputer)",
      "categoricas": "Constante 'DESCONHECIDO' (SimpleImputer)"
    },
    "normalizacao": "StandardScaler (media=0, std=1)",
    "encoding": "OrdinalEncoder (evita explosao de dimensao)",
    "features_numericas": [
      "hora_dia",
      "dia_semana",
      "mes_ano",
      "fim_de_semana",
      "alta_temporada",
      "media_atraso_empresa",
      "media_atraso_origem",
      "media_atraso_destino"
    ],
    "features_categoricas": [
      "empresa_

In [ ]:
# ============================================
# RESUMO FINAL - ENTREGA SEMANA 2
# ============================================

print("\n" + "="*80)
print("RESUMO FINAL - DS1 + DS2")
print("="*80)

print("\nOBJETIVOS CUMPRIDOS:")
print("\nDS1 (Pre-processamento):")
print("  - Amostra estratificada (10%) criada")
print("  - Tratamento de nulos implementado")
print("  - Normalizacao (StandardScaler) aplicada")
print("  - Encoding (OrdinalEncoder) aplicado")
print("  - Pipeline reutilizavel criado")

print("\nDS2 (Feature Engineering):")
print("  - Features temporais criadas (6 novas features)")
print("  - Features agregadas criadas (3 medias)")
print("  - MediaAtrasoTransformer implementado")
print("  - Solucao para problema do groupby() na API")

print("\nARQUIVOS ENTREGUES:")
print("  1. media_transformer_ds2.pkl")
print("  2. preprocessor_ds1.pkl")
print("  3. X_train_processed.npy")
print("  4. y_train.npy")
print("  5. documentacao_ds1_ds2.json")
print("  6. S02_DS1_DS2_Pipeline.ipynb")

print("\nPROXIMOS PASSOS:")
print("  - DS3: Carregar X_train_processed.npy e y_train.npy")
print("  - DS3: Treinar modelo baseline")
print("  - DS5: Integrar pipelines na API")
print("  - DS5: Testar endpoint /predict")

print("\nAPRESENTACAO (Quinta 25/12):")
print("  1. Mostrar problema do groupby() na API")
print("  2. Explicar MediaAtrasoTransformer (solucao)")
print("  3. Demonstrar pipeline completo funcionando")
print("  4. Mostrar teste com novo voo (celula 14)")

print("\n" + "="*80)
print("SEMANA 2 (DS1 + DS2) CONCLUIDA COM SUCESSO")
print("="*80)



RESUMO FINAL - DS1 + DS2

OBJETIVOS CUMPRIDOS:

DS1 (Pre-processamento):
  - Amostra estratificada (10%) criada
  - Tratamento de nulos implementado
  - Normalizacao (StandardScaler) aplicada
  - Encoding (OrdinalEncoder) aplicado
  - Pipeline reutilizavel criado

DS2 (Feature Engineering):
  - Features temporais criadas (6 novas features)
  - Features agregadas criadas (3 medias)
  - MediaAtrasoTransformer implementado
  - Solucao para problema do groupby() na API

ARQUIVOS ENTREGUES:
  1. media_transformer_ds2.pkl
  2. preprocessor_ds1.pkl
  3. X_train_processed.npy
  4. y_train.npy
  5. documentacao_ds1_ds2.json
  6. S02_DS1_DS2_Pipeline.ipynb

PROXIMOS PASSOS:
  - DS3: Carregar X_train_processed.npy e y_train.npy
  - DS3: Treinar modelo baseline
  - DS5: Integrar pipelines na API
  - DS5: Testar endpoint /predict

APRESENTACAO (Quinta 25/12):
  1. Mostrar problema do groupby() na API
  2. Explicar MediaAtrasoTransformer (solucao)
  3. Demonstrar pipeline completo funcionando
  4. 